In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import seaborn as sns
import sklearn as sk
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [3]:
games = pd.read_csv('games_new_vars.csv')

Here, we will fit a few logistic regression models to predict the probability of a higher rating player winning. 
Order of business: 

1) Perform k-fold cross validation (k = 5) for a simple logistic regression. Predictor will be absolute value of difference (abs_diff_rating) 
in rating, and response will be whether the higher rating won (hr_won). 

2) Perform k-fold cross validation (k = 5) for a multiple logistic regression. We will include predictors that might be relevant. For example. 

In [4]:
# Empty model
lg = LogisticRegression()

In [10]:
# Perform k-fold cross validation for k = 5
kf_five = KFold(n_splits = 5)

# We will find the estimated_mse
est_mse_five = 0
for train, test in kf_five.split(games):
    
    # Create training and test data
    train_data = games.iloc[train][['abs_diff_rating', 'higher_rating_won']]
    test_data = games.iloc[test][['abs_diff_rating', 'higher_rating_won']]
    
    # Predictor and response (training)
    X_train = DataFrame(train_data.abs_diff_rating)
    Y_train = DataFrame(train_data.higher_rating_won)
    
    # Fit the model
    lg.fit(X_train, Y_train)
    
    # Predictor and response(test)
    X_test = DataFrame(test_data.abs_diff_rating)
    Y_test = DataFrame(test_data.higher_rating_won)
    
    # Store the score for the model
    est_mse_five += lg.score(X_test, Y_test)
    
# Take the average MSE
est_mse_five = est_mse_five / 5

/Users/Leonard/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Leonard/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Leonard/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Leonard/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

In [50]:
trya = games.columns.get_indexer(['diff_rating', 'abs_diff_rating'])
tryb = games.columns.get_indexer(['abs_diff_rating'])
full = list(trya) + list(tryb)
type(full)

list

In [77]:
type(games['diff_rating'])

pandas.core.series.Series

In [60]:
games.head(5)

,Unnamed: 0,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,...,black_rating,moves,opening_eco,opening_name,opening_ply,diff_rating,abs_diff_rating,higher_rating,higher_rating_won,higher_rating_coded
0,0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,...,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5,309,309,white,1,1
1,1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,...,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4,61,61,white,0,1
2,2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,...,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3,-4,4,black,0,0
3,3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,...,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3,-15,15,black,0,0
4,4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,...,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5,54,54,white,1,1


In [27]:
games.iloc[1][[17, 6]]

diff_rating           61
victory_status    resign
Name: 1, dtype: object

In [77]:
# Look at the average MSE
est_mse_five

0.62483754172536

This metric tells me that the model will properly predict a win or los 62% of the time. 

In [78]:
# Fit a multiple logistic regression model 
lg_mult = LogisticRegression()

In [87]:
X = DataFrame(games[['abs_diff_rating', 'higher_rating_coded', 'turns']])
Y = DataFrame(games['hr_won'])

In [88]:
lg_mult.fit(X, Y)

/Users/Leonard/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [89]:
lg_mult.score(X, Y)

0.6358061621298235